In [27]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer

In [12]:
df = pd.read_table("../data/35509-0001-Data.tsv")
df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,99,99,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,99,99,14,9999,99,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,99,99,14,9999,99,1,2,...,1,99,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,99,99,16,9999,99,4,93,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,99,99,14,9999,99,4,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,99,99,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,99,99,16,9999,99,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,99,99,991,9991,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,991,9991,91,91,91,...,9,99,9,9,1,1,2,3104.06,30051,2


In [13]:
df = df.replace({9999: np.nan, 99: np.nan, 999: np.nan, 98: np.nan, 998: np.nan, 9998: np.nan, 97: np.nan, 997: np.nan, 9997: np.nan})
df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1.0,48694667,1,NaN,NaN,19.0,2012.0,7.0,1,1.0,...,1,4.0,1,1,2,2,2,4398.40,30017,1
1,2.0,88530883,1,NaN,NaN,14.0,NaN,NaN,2,93.0,...,1,4.0,1,1,1,1,2,1419.19,30052,2
2,3.0,33251077,1,NaN,NaN,14.0,NaN,NaN,1,2.0,...,1,NaN,9,9,3,3,2,14052.62,30028,1
3,4.0,37814127,1,NaN,NaN,16.0,NaN,NaN,4,93.0,...,1,4.0,1,1,1,1,2,10848.18,30055,2
4,5.0,18762590,1,NaN,NaN,14.0,NaN,NaN,4,93.0,...,1,1.0,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156.0,13675473,2,NaN,NaN,991.0,9991.0,91.0,91,91.0,...,1,2.0,1,1,3,3,2,679.36,30003,1
55156,55157.0,49609908,1,NaN,NaN,16.0,NaN,NaN,4,93.0,...,1,1.0,1,1,2,2,2,2296.28,30057,2
55157,55158.0,81795924,2,NaN,NaN,991.0,9991.0,91.0,91,91.0,...,1,4.0,1,1,3,3,2,17180.64,30026,1
55158,55159.0,17198338,2,4.0,4.0,991.0,9991.0,91.0,91,91.0,...,9,NaN,9,9,1,1,2,3104.06,30051,2


In [14]:
columns_nas = df.isna().sum()/df.shape[0]
columns_nas

CASEID      0.000163
QUESTID2    0.000000
CIGEVER     0.000000
CIGOFRSM    0.733611
CIGWILYR    0.733539
              ...   
COUTYP2     0.000000
MAIIN002    0.000000
ANALWT_C    0.000000
VESTR       0.000000
VEREP       0.000000
Length: 3141, dtype: float64

In [18]:
columns_nas[columns_nas > 0.9]

ALTOTFG     0.994743
ALFQFLG     0.985334
ALDYSFG     0.999656
MRTOTFG     0.994507
MJFQFLG     0.992059
              ...   
LFSCHWH2    0.962273
LFTSCHA2    0.962255
WRKIDSY2    0.903318
WRKOCUY2    0.903354
WRKBZCY2    0.903354
Length: 536, dtype: float64

HLTINDRG: represents drug abuse

In [37]:
X = df.iloc[:, 2:]
X

,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,NaN,NaN,19.0,2012.0,7.0,1,1.0,NaN,2.0,...,1,4.0,1,1,2,2,2,4398.40,30017,1
1,1,NaN,NaN,14.0,NaN,NaN,2,93.0,93.0,93.0,...,1,4.0,1,1,1,1,2,1419.19,30052,2
2,1,NaN,NaN,14.0,NaN,NaN,1,2.0,NaN,1.0,...,1,NaN,9,9,3,3,2,14052.62,30028,1
3,1,NaN,NaN,16.0,NaN,NaN,4,93.0,93.0,93.0,...,1,4.0,1,1,1,1,2,10848.18,30055,2
4,1,NaN,NaN,14.0,NaN,NaN,4,93.0,93.0,93.0,...,1,1.0,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,2,NaN,NaN,991.0,9991.0,91.0,91,91.0,91.0,91.0,...,1,2.0,1,1,3,3,2,679.36,30003,1
55156,1,NaN,NaN,16.0,NaN,NaN,4,93.0,93.0,93.0,...,1,1.0,1,1,2,2,2,2296.28,30057,2
55157,2,NaN,NaN,991.0,9991.0,91.0,91,91.0,91.0,91.0,...,1,4.0,1,1,3,3,2,17180.64,30026,1
55158,2,4.0,4.0,991.0,9991.0,91.0,91,91.0,91.0,91.0,...,9,NaN,9,9,1,1,2,3104.06,30051,2


In [38]:
imputer = SimpleImputer(strategy='mean')
imputed_data = imputer.fit_transform(X)

In [39]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(imputed_data)

In [40]:
scaled_data

array([[-1.01564079e+00,  5.02811075e-16, -3.45160597e-16, ...,
        -4.46895310e-02, -7.62604161e-01, -1.01453639e+00],
       [-1.01564079e+00,  5.02811075e-16, -3.45160597e-16, ...,
        -4.16052913e-01,  1.24921793e+00,  9.85671890e-01],
       [-1.01564079e+00,  5.02811075e-16, -3.45160597e-16, ...,
         1.15872473e+00, -1.30317219e-01, -1.01453639e+00],
       ...,
       [ 9.84600073e-01,  5.02811075e-16, -3.45160597e-16, ...,
         1.54863752e+00, -2.45278481e-01, -1.01453639e+00],
       [ 9.84600073e-01,  1.47866418e-01,  5.29330385e-02, ...,
        -2.06031122e-01,  1.19173730e+00,  9.85671890e-01],
       [ 9.84600073e-01,  5.02811075e-16, -3.45160597e-16, ...,
        -1.89959787e-01,  3.87008461e-01,  9.85671890e-01]])

In [41]:
pca = PCA()
pca.fit(scaled_data)

PCA()

In [42]:
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

variance_threshold = 0.95  # Set your desired threshold here
n_components = np.argmax(cumulative_variance_ratio >= variance_threshold) + 1

In [43]:
n_components

889

only 889 components contribute to the variance. This variance could be skewed due to the 99,999, or 9999 values that aren't considered missing

In [46]:
loadings = pca.components_
absolute_loadings = np.abs(loadings)
feature_importance = np.mean(absolute_loadings, axis=0)
sorted_indices = np.argsort(feature_importance)[::-1]
feature_names = list(X.columns)
for i in sorted_indices:
    print(f"{feature_names[i]}: {feature_importance[i]}")

HRDAYPYR: 0.012606906666590754
HRDAYPWK: 0.012460260326903491
STMYRTOT: 0.012455226589542503
METHREC: 0.012362652849074908
OXFQFLG: 0.01233662390543711
II2MJRC: 0.012279989182312557
PRBSTWAY: 0.01219858334681275
II2COCRC: 0.012160459552642658
CRDAYPWK: 0.012113889801775078
MTTOTFG: 0.012100550858758845
CRDAYPYR: 0.012065990219424338
II2ALCRC: 0.011993387327964115
TRTOTFG: 0.011971248567646198
CCYRAVE: 0.011964638755394979
IITRNRC: 0.01195303899559264
STTOTFG: 0.011896221171068477
STFQFLG: 0.011893893755470751
II2OXYRC: 0.011884985840339955
II2TRNRC: 0.01187104594779165
HR30EST: 0.011839748446853572
IRCOCFM: 0.011835995580020936
IIECSRC: 0.011826279533464976
MTHYRTOT: 0.011820028169504195
II2ECSRC: 0.011796239630262361
IIOXYRC: 0.011792576468697188
PRDAYPMO: 0.011756031515998668
PRDAYPYR: 0.011753339765343871
OXBSTWAY: 0.011738501292617729
SEDYRTOT: 0.01173725941702527
HER30USE: 0.011726577938029219
COCMON: 0.011715038167662561
HRDAYPMO: 0.011656917841589787
SEDMFU: 0.011640913628279272

ANLPHCTD: 0.008247436787896549
YUHOANGR: 0.00824290713407667
MJPRIOR: 0.008242566456630409
RKTRYLSD: 0.008242310272880297
MMGGIVE: 0.00824206718120874
TXRNNTSP: 0.008240285631803915
FUCGR21: 0.008240223928414475
NDTMRHER: 0.008239907279193015
MTHFRBUY: 0.008238954194078142
STMRX1D2: 0.00823881588257725
YEYHGUN: 0.008237458645227774
MTHSRCF2: 0.008237199661165925
IIMEDICR: 0.008235811818048534
IRANLYFU: 0.008235378293631662
DSTCHR30: 0.008234908101563531
SEDNEWA: 0.008233319025879609
IREDUC2: 0.008233162424497347
SEDSRCFR: 0.00823107149674412
STMSTOLE: 0.008230644620397694
AUALHLIN: 0.00822653746781381
USEALMJ: 0.00822651582481766
RSKMJREG: 0.008226186567571157
ANLSRCF2: 0.008225890079001801
IRINHYFU: 0.008225813383099493
AUOPOTOP: 0.008224154265725862
UADPEOP: 0.00822373745518351
STMCUT1X: 0.008222664523498598
TRNFFST2: 0.008222011865080914
AUOPDOC: 0.00822116570632222
ADWRSTHK: 0.008220636377396085
ADWRGROW: 0.00822063637738568
IMPSOCM: 0.008219411978618345
SOMHBRK: 0.0082188308687878

In [49]:
X['SUMFLAG']

0        0
1        1
2        0
3        0
4        0
        ..
55155    0
55156    1
55157    0
55158    0
55159    1
Name: SUMFLAG, Length: 55160, dtype: int64

In [50]:
correlation_matrix =X.corr()
correlation_with_sumflag = correlation_matrix["SUMFLAG"]
correlation_with_sumflag

CIGEVER    -0.521806
CIGOFRSM   -0.034849
CIGWILYR    0.009799
CIGTRY     -0.520907
CIGYFU     -0.254350
              ...   
COUTYP2    -0.012244
MAIIN002   -0.011107
ANALWT_C    0.035440
VESTR      -0.005854
VEREP       0.000376
Name: SUMFLAG, Length: 3139, dtype: float64

In [52]:
most_correlated_features = abs(correlation_with_sumflag).drop(["SUMFLAG"]).sort_values(ascending = False)
most_correlated_features

SUMAGE      0.999977
TXYRSGAD    0.998775
NDTXMJR     0.998431
NDTXHALR    0.998075
NDTXCOCR    0.997917
              ...   
TXALONEV         NaN
TXALONAG         NaN
HLCALLFG         NaN
HLCALL99         NaN
MOVYRFGR         NaN
Name: SUMFLAG, Length: 3138, dtype: float64